#### Script para subir os dados pras tabelas no PostgreSQL

Projeto: Sistema de Apoio à Caracterização de Imóveis Rurais  
Embrapa/2023

In [ ]:
# Imports
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import pandas as pd
import geopandas as gpd
import glob
import os

In [ ]:
# Definir aos parâmetros do banco de dados
host = "localhost:5438"
database = "lair"
user = "postgres"
password = "postgres"

In [ ]:
# Conexão com o banco de dados
connection_string = f"postgresql://{user}:{password}@{host}/{database}"
engine = create_engine(connection_string)

In [ ]:
# Definir o caminho da base de dados que irá pro banco de dados
base_path = r"C:\Users\youx\Documents\Embrapa\Teste_Notebooks\Usos_ABC\Usos_ABC_Municipios\2021"

In [ ]:
# Definir qual o schema e table de acordo com a base de dados
table = '2021'
schema = 'Usos_ABC'

In [ ]:
# Lendo o caminho dos shapefiles
shapefiles = glob.glob(base_path + f'**/*.shp')
for shape in shapefiles:
    gdf = gpd.read_file(shape)
    gdf = gdf.to_crs(epsg=4326)
    gdf = gdf.explode(ignore_index=True)
    gdf.columns = [x.lower() for x in gdf.columns]
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
    gdf.drop(['geometry'], axis=1, inplace=True)
    gdf.to_sql(f'{table}', engine, schema= f'{schema}', if_exists='append', index=False, 
                         dtype={'geom': Geometry('POLYGON', srid= 4326)})